In [ ]:
import pandas as pd
import os

file_path = os.path.join("Data", "Alex_Activities.csv")
alex = pd.read_csv(file_path)

alex["Person"] = "Alex"

columns_to_keep = [
    "Activity Date", "Activity Type", "Elapsed Time", "Distance", "Moving Time", "Max Speed",
    "Average Speed", "Elevation Gain", "Elevation Loss", "Elevation Low", "Elevation High", "Max Grade",
    "Average Grade", "Average Grade Adjusted Pace", "Person"]
alex = alex[columns_to_keep]

alex = alex[alex["Activity Type"] == "Run"]

# ------- FORMAT DATE (e.g. 12/1/23) --------
alex["Activity Date"] = pd.to_datetime(alex["Activity Date"])
alex["Activity Date"] = alex["Activity Date"].dt.strftime("%-m/%-d/%y")

# ------- TIME FORMAT (seconds → M:SS) --------
def seconds_to_mmss(value):
    value = float(value)
    minutes = int(value) // 60
    seconds = int(value) % 60
    return f"{minutes}:{seconds:02d}"

alex["Elapsed Time"] = alex["Elapsed Time"].apply(seconds_to_mmss)
alex["Moving Time"] = alex["Moving Time"].apply(seconds_to_mmss)

# ------- DISTANCE (km → miles) --------
alex["Distance"] = (alex["Distance"] * 0.621371).round(2)

# ------- SPEEDS (m/s → mph) --------
alex["Max Speed"] = (alex["Max Speed"] * 2.23694).round(2)
alex["Average Speed"] = (alex["Average Speed"] * 2.23694).round(2)
alex["Average Grade Adjusted Pace"] = (
    alex["Average Grade Adjusted Pace"] * 2.23694
).round(2)

# ------- ELEVATION (m → ft) --------
meters_to_feet = 3.28084
elevation_cols = ["Elevation Gain", "Elevation Loss", "Elevation Low", "Elevation High"]
alex[elevation_cols] = (alex[elevation_cols] * meters_to_feet).round(1)

# ------- PACE COLUMNS (mph → min/mile, formatted M:SS) --------
def mph_to_pace(mph):
    if pd.isna(mph) or mph <= 0:
        return None
    pace = 60 / mph
    minutes = int(pace)
    seconds = int(round((pace - minutes) * 60))
    if seconds == 60:
        minutes += 1
        seconds = 0
    return f"{minutes}:{seconds:02d}"

alex["Pace"] = alex["Average Speed"].apply(mph_to_pace)
alex["Grade Adjusted Pace"] = alex["Average Grade Adjusted Pace"].apply(mph_to_pace)

desired_order = ["Activity Date", "Activity Type", "Elapsed Time", "Distance", "Pace", 
    "Moving Time", "Max Speed", "Average Speed", "Elevation Gain", "Elevation Loss",
    "Elevation Low", "Elevation High", "Max Grade", "Average Grade", "Average Grade Adjusted Pace",
    "Grade Adjusted Pace", "Person"]

alex = alex[desired_order]

alex.reset_index(drop=True, inplace=True)
alex

/var/folders/xb/jy20y8px72jfxywp5jks_bnc0000gn/T/ipykernel_64057/3085962537.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  alex["Activity Date"] = pd.to_datetime(alex["Activity Date"])


,Activity Date,Activity Type,Elapsed Time,Distance,Pace,Moving Time,Max Speed,Average Speed,Elevation Gain,Elevation Loss,Elevation Low,Elevation High,Max Grade,Average Grade,Average Grade Adjusted Pace,Grade Adjusted Pace,Person
0,12/1/23,Run,12:26,1.68,7:22,12:26,10.87,8.14,0.0,0.0,1.0,5.6,1.7,0.0,NaN,None,Alex
1,1/11/24,Run,13:57,1.76,7:56,13:57,10.56,7.57,9.5,0.0,0.3,10.2,3.3,0.1,NaN,None,Alex
2,2/19/24,Run,15:27,2.16,7:08,15:27,11.23,8.41,6.6,392.4,11.2,398.0,5.0,-3.4,NaN,None,Alex
3,3/5/24,Run,17:40,2.50,7:03,17:40,12.48,8.51,0.0,394.4,11.2,405.5,9.4,-2.9,NaN,None,Alex
4,3/10/24,Run,14:34,2.08,7:01,14:34,13.06,8.56,6.6,400.3,11.5,405.5,8.6,-3.5,NaN,None,Alex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11/8/25,Run,60:42,5.48,7:29,41:03,17.72,8.02,278.2,277.6,229.3,364.5,12.2,0.0,8.02,7:29,Alex
124,11/12/25,Run,50:08,5.33,7:33,40:17,11.05,7.94,351.0,352.4,273.0,388.1,26.3,0.0,8.10,7:24,Alex
125,11/20/25,Run,44:52,4.16,7:29,31:09,11.63,8.02,150.6,150.9,298.9,367.1,11.5,0.0,8.10,7:24,Alex
126,11/21/25,Run,63:16,6.46,7:44,49:58,13.24,7.76,285.8,270.0,327.1,444.6,12.6,0.0,7.85,7:39,Alex
